# Spark Structured Streaming Project

Project contains Apache Spark Streaming. Apache Spark write streams to postgre sql, apache hive and deltalake with processing read stream dataframe.

## Streaming CSV to Folder

In [17]:
# python dataframe_to_log.py -idx True -i input/iot_telemetry_data.csv

As the above, run python file at console. This provides streaming csv to folder.

## Import Libraries & SparkSession

In [1]:
from pyspark.sql import SparkSession, functions as F

import findspark
from spark_utils import *

findspark.init("C:\Program Files\Spark\spark-3.3.1-bin-hadoop3")

In [2]:
spark = (SparkSession.builder
        .master("yarn")
        .appName("Spark Streaming with Multiple Sink")
        .config("spark.jars.packages", "io.delta:delta-core_2.12:2.2.0") 
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") 
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") 
        .config("spark.sql.adaptive.enabled", True)
        .enableHiveSupport()
        .getOrCreate())

Create spark session with deltalake and apache hive configurations. After that import to delta library.

In [3]:
from delta.tables import *

## IOT Schema

In [4]:
iot_schema = "row_id int, " \
    "ts double, " \
    "device string, " \
    "co float, " \
    "humidity float, " \
    "light boolean, " \
    "lpg float, " \
    "motion boolean, " \
    "smoke float, " \
    "temp float, " \
    "time timestamp"

Make schema for dataframe on the above because spark can read data types wrongly in order that set manually to schema.

In [4]:
file_path = "file:///Users/talha/OneDrive/Masaüstü/Talha Nebi Kumru/Data Enginnering/Miuul/RealTimeDPWithSpark/Spark_Streaming_with_Multiple_Sink/output"
delta_path = "/user/talha/datasets/delta-stream"

## Read Stream

In [6]:
stream_df = (spark.readStream
            .format("csv")
            .schema(iot_schema)
            .option("header", True)
            .option("maxFilesPerTrigger", 1)
            .load(file_path))

## Multiple Sink

In [8]:
def write_multiple(df, batchID):
    df.persist()
    
    print(df.limit(5).toPandas().head())
    
    # Write to PostgreSQL
    write_psql(df.filter(" device == '00:0f:00:70:91:0a' "), 
               table_name="sensor_0a", 
               password="password")
    
    # Write to Apache Hive
    write_hive(df.filter(" device == 'b8:27:eb:bf:9d:51' "), 
               mode="append", 
               table_name="sensor_51")
    
    # Write to Deltalake
    write_deltalake(df.filter(" device == '1c:bf:ce:15:ec:4d' "), 
                    mode="append", 
                    path=delta_path)
    
    df.unpersist()

This function provides appending dataframe to psql, hive and deltalake.

## Write Stream

In [9]:
streaming_query = (
    stream_df.writeStream
        .foreachBatch(write_multiple)
        .option("checkpointLocation", "checkpoint")
        .start()
)
streaming_query.awaitTermination(100)

C:\Users\talha\anaconda3\lib\site-packages\pyspark\sql\pandas\conversion.py:194: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


   row_id            ts  ...       temp                       time
0       1  1.594512e+09  ...  19.700001 2023-02-13 01:56:59.308220
1       2  1.594512e+09  ...  22.600000 2023-02-13 01:56:59.823688
2       3  1.594512e+09  ...  27.000000 2023-02-13 01:57:00.337171
3       4  1.594512e+09  ...  22.600000 2023-02-13 01:57:00.851784
4       5  1.594512e+09  ...  27.000000 2023-02-13 01:57:01.359949

[5 rows x 11 columns]


C:\Users\talha\anaconda3\lib\site-packages\pyspark\sql\pandas\conversion.py:194: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


   row_id            ts  ...       temp                       time
0      11  1.594512e+09  ...  27.000000 2023-02-13 01:57:04.426730
1      12  1.594512e+09  ...  22.600000 2023-02-13 01:57:04.939046
2      13  1.594512e+09  ...  27.000000 2023-02-13 01:57:05.440151
3      14  1.594512e+09  ...  22.600000 2023-02-13 01:57:05.955877
4      15  1.594512e+09  ...  19.700001 2023-02-13 01:57:06.468947

[5 rows x 11 columns]


C:\Users\talha\anaconda3\lib\site-packages\pyspark\sql\pandas\conversion.py:194: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


...

C:\Users\talha\anaconda3\lib\site-packages\pyspark\sql\pandas\conversion.py:194: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


   row_id            ts  ...       temp                       time
0     331  1.594513e+09  ...  27.000000 2023-02-13 01:59:48.094689
1     332  1.594513e+09  ...  22.400000 2023-02-13 01:59:48.597005
2     333  1.594513e+09  ...  19.700001 2023-02-13 01:59:49.112011
3     334  1.594513e+09  ...  22.400000 2023-02-13 01:59:49.625096
4     335  1.594513e+09  ...  27.000000 2023-02-13 01:59:50.140641

[5 rows x 11 columns]


C:\Users\talha\anaconda3\lib\site-packages\pyspark\sql\pandas\conversion.py:194: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


   row_id            ts  ...  temp                       time
0     341  1.594513e+09  ...  22.4 2023-02-13 01:59:53.193341
1     342  1.594513e+09  ...  22.4 2023-02-13 01:59:53.705529
2     343  1.594513e+09  ...  27.0 2023-02-13 01:59:54.218743
3     344  1.594513e+09  ...  22.4 2023-02-13 01:59:54.730727
4     345  1.594513e+09  ...  19.6 2023-02-13 01:59:55.244762

[5 rows x 11 columns]


ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "C:\Users\talha\anaconda3\lib\site-packages\py4j\clientserver.py", line 467, in send_command
    self.socket.sendall(command.encode("utf-8"))
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\talha\anaconda3\lib\site-packages\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "C:\Users\talha\anaconda3\lib\site-packages\py4j\clientserver.py", line 470, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending
ERROR:py4j.clientserver:There was an exception while executing the Python Proxy on the Python Side.
Traceback (most recent call last):
  File "C:\Users\talha\anaconda3\lib\site-packages\py4j\clientserver.py", line 581, in _call_proxy
   

## Control

Control to is saved data in psql, hive and deltalake.

In [13]:
psql_df = read_psql(spark, table_name="sensor_0a", password="password")

In [15]:
psql_df.limit(5).toPandas()

C:\Users\talha\anaconda3\lib\site-packages\pyspark\sql\pandas\conversion.py:194: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,row_id,ts,device,co,humidity,light,lpg,motion,smoke,temp,time
0,1,1.594512e+09,00:0f:00:70:91:0a,0.002840,76.000000,False,0.005114,False,0.013275,19.700001,2023-02-13 01:56:59.308220
1,7,1.594512e+09,00:0f:00:70:91:0a,0.002938,76.000000,False,0.005241,False,0.013628,19.700001,2023-02-13 01:57:02.388916
2,15,1.594512e+09,00:0f:00:70:91:0a,0.002905,75.800003,False,0.005199,False,0.013509,19.700001,2023-02-13 01:57:06.468947
3,23,1.594512e+09,00:0f:00:70:91:0a,0.002840,76.000000,False,0.005114,False,0.013275,19.700001,2023-02-13 01:57:10.538998
4,25,1.594512e+09,00:0f:00:70:91:0a,0.002840,76.000000,False,0.005114,False,0.013275,19.799999,2023-02-13 01:57:11.567682


In [9]:
hive_df = read_hive(spark, db_name="test", table_name="sensor_51")

In [12]:
hive_df.limit(5).toPandas()

C:\Users\talha\anaconda3\lib\site-packages\pyspark\sql\pandas\conversion.py:194: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,row_id,ts,device,co,humidity,light,lpg,motion,smoke,temp,time
0,341,1.594513e+09,b8:27:eb:bf:9d:51,0.004973,49.200001,False,0.007670,False,0.020466,22.400000,2023-02-13 01:59:53.193341
1,342,1.594513e+09,b8:27:eb:bf:9d:51,0.004988,49.200001,False,0.007686,False,0.020512,22.400000,2023-02-13 01:59:53.705529
2,344,1.594513e+09,b8:27:eb:bf:9d:51,0.004972,49.299999,False,0.007668,False,0.020461,22.400000,2023-02-13 01:59:54.730727
3,346,1.594513e+09,b8:27:eb:bf:9d:51,0.004988,49.200001,False,0.007686,False,0.020512,22.299999,2023-02-13 01:59:55.758815
4,348,1.594513e+09,b8:27:eb:bf:9d:51,0.004982,49.299999,False,0.007680,False,0.020493,22.400000,2023-02-13 01:59:56.782545


In [5]:
delta_df = read_delta(spark, delta_path)

In [7]:
delta_df.limit(5).toPandas()

C:\Users\talha\anaconda3\lib\site-packages\pyspark\sql\pandas\conversion.py:194: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,row_id,ts,device,co,humidity,light,lpg,motion,smoke,temp,time
0,343,1.594513e+09,1c:bf:ce:15:ec:4d,0.004342,77.599998,True,0.006952,False,0.018427,27.0,2023-02-13 01:59:54.218743
1,347,1.594513e+09,1c:bf:ce:15:ec:4d,0.004291,77.599998,True,0.006894,False,0.018261,27.0,2023-02-13 01:59:56.273850


## Stop

In [16]:
spark.stop()